<a href="https://colab.research.google.com/github/Luca-Wiehe/nerf_segmentation/blob/main/open_nerf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os

# set path to project folder
gdrive_path='/content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/nerf_segmentation/' # Luca's Path
#gdrive_path='/content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/' # Luis' Path

# mount Google Drive
drive.mount('/content/gdrive', force_remount=True)

# navigate to Google Drive folder
os.chdir(gdrive_path)

# check that we are in the right folder
print(sorted(os.listdir()))

Mounted at /content/gdrive
['.git', 'README.md', 'data', 'data_loader.ipynb', 'data_loading', 'example_arrows', 'gitignore', 'open_nerf.ipynb']


Folder Structure in Colab:
```
adl4vc  
|_ nerf_segmentation  
    |_ data  
        |_ lerf  
        |_ replica  
|_ opennerf
```



Installing Requirements, building wheels for tinycudann takes a while (for me ~19 min)

In [2]:
!python --version
!nvcc --version
!uname -a

Python 3.10.12
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0
Linux d52b40d3f71f 6.1.58+ #1 SMP PREEMPT_DYNAMIC Sat Nov 18 15:31:17 UTC 2023 x86_64 x86_64 x86_64 GNU/Linux


In [2]:
!python -m pip install --upgrade pip
!python -m pip install pytorch==2.1.2 torchvision==0.16.2 torchaudio==2.1.2 pytorch-cuda=12.1 -c pytorch -c nvidia
!python -m pip install ninja git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'pytorch'
  Cloning https://github.com/NVlabs/tiny-cuda-nn/ to /tmp/pip-req-build-xqqml1df
  Running command git clone --filter=blob:none --quiet https://github.com/NVlabs/tiny-cuda-nn/ /tmp/pip-req-build-xqqml1df
  Resolved https://github.com/NVlabs/tiny-cuda-nn/ to commit 2ec562e853e6f482b5d09168705205f46358fb39
  Running command git submodule update --init --recursive -q
  Preparing metadata (setup.py) ... done
  Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl.metadata (5.3 kB)
Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
  Created wheel for tinycudann: filename=tinycudann-1.7-cp310-cp310-linux_x86_64.whl size=30358077 sha256=46ecee7d1fa958a5ab27e7506ce248c8b0bb1d051859aae331d475f3694fc908
  Stored in directory: /tmp/pip-ephem-wheel-cache-xga4bbkg/wheels/32/d8/5e/dc94eca0794af9e09a6d97f19cf15dfe9bbbc4d56ae4db4aa2
Successful

In [3]:
%%capture
!python -m pip install open3d
!python -m pip install nerfstudio
!python -m pip install pyviz3d

In [4]:
%%capture
!python -m pip install -e /content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/opennerf
#!python -m pip install -e /content/gdrive/MyDrive/Uni/adl4vc/opennerf

In [5]:
!ns-install-cli

[20:12:23] 🤷 .zshrc not found, skipping.                                                                 ]8;id=142336;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/scripts/completions/install.py\install.py]8;;\:]8;id=112615;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/scripts/completions/install.py#369\369]8;;\
           🔍 Found .bashrc!                                                                              ]8;id=866896;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/scripts/completions/install.py\install.py]8;;\:]8;id=827972;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/scripts/completions/install.py#371\371]8;;\
[20:12:26] ✔ Wrote new completion to                                                                      ]8;id=971561;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/scripts/completions/install.py\install.py]8;;\:]8;id=131005;file:///usr/local/lib/python3.10/dist-packages/nerfstudio/scripts/

# Methods from OpenNerf Repo

# Visualizing Replica Dataset

Download Replica dataset from wget https://cvg-data.inf.ethz.ch/nice-slam/data/Replica.zip



In [ ]:
scenes = ['office0'] #, 'office1', 'office2', 'office3', 'office4', 'room0', 'room1', 'room2',]

In [ ]:
%%capture
from data_loading.replica import process_replica

for scene in scenes:
    data = gdrive_path + f'data/replica/{scene}'
    output_dir = gdrive_path + f'data/nerfstudio_/replica_{scene}'
    process_replica(Path(data), Path(output_dir))

In [6]:
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(6008)"))

https://69k6v7ih65w-496ff2e9c6d22116-6008-colab.googleusercontent.com/


In [7]:
!cd /content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/nerf_segmentation/data/nerfstudio_/replica_office0/visualization && python -m http.server 6008
# !cd /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/nerfstudio_/replica_office0/visualization && python -m http.server 6008

Serving HTTP on 0.0.0.0 port 6008 (http://0.0.0.0:6008/) ...
127.0.0.1 - - [09/May/2024 20:14:47] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:14:48] "GET /css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:14:48] "GET /js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:14:48] "GET /js/scene.js HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:14:48] "GET /js/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:15:08] "GET /nodes.json HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:15:09] "GET /favicon.ico HTTP/1.1" 200 -

Keyboard interrupt received, exiting.
^C


In [8]:
!cd /content/gdrive/MyDrive/1-university/masters/2-semester/in2390_adl4cv/nerf_segmentation/example_arrows && python -m http.server 6008
#!cd /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/example_arrows && python -m http.server 6008 # Luis' path

Serving HTTP on 0.0.0.0 port 6008 (http://0.0.0.0:6008/) ...
127.0.0.1 - - [09/May/2024 20:15:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:15:32] "GET /css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:15:32] "GET /js/scene.js HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:15:32] "GET /js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:15:32] "GET /js/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:15:33] "GET /nodes.json HTTP/1.1" 200 -
127.0.0.1 - - [09/May/2024 20:15:34] "GET /scene.bin HTTP/1.1" 200 -

Keyboard interrupt received, exiting.
^C


# Visualizing LERF Dataset

The following command will generate a lerf visualization. This is saved in the stated directory. I havent figured out how to access a "local google colab server" from the colab notebook but to visualize it you can just download the "visualisation" subdir and execute the commands in you local terminal

Download the LERF Dataset from: https://drive.google.com/drive/folders/1vh0mSl7v29yaGsxleadcj-LCZOE_WEWB

In [ ]:
from data_loading.lerf import visualize_lerf_trajector

output_dir = '/content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/nerfstudio_/replica_office0'
visualize_lerf_trajector(Path(output_dir))

In [ ]:
lerf_dir = '/content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/lerf/bouquet'
visualize_lerf_trajector(Path(lerf_dir))


************************************************************************
1) Start local server:
    cd /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/lerf/bouquet/visualization; python -m http.server 6008
2) Open in browser:
    http://localhost:6008
************************************************************************


This is the attempt to access the visualisation in colab, doesnt work unfortunately

In [ ]:
!cd /content/gdrive/MyDrive/Uni/adl4vc/nerf_segmentation/data/lerf/bouquet/visualization && python -m http.server 6008

Serving HTTP on 0.0.0.0 port 6008 (http://0.0.0.0:6008/) ...
127.0.0.1 - - [03/May/2024 08:50:37] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 08:50:39] "GET /js/bootstrap.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 08:50:39] "GET /js/scene.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 08:50:40] "GET /css/bootstrap.min.css HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 08:50:40] "GET /js/jquery.min.js HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 08:50:41] "GET /nodes.json HTTP/1.1" 200 -
127.0.0.1 - - [03/May/2024 08:50:41] "GET /favicon.ico HTTP/1.1" 200 -

Keyboard interrupt received, exiting.


http://localhost:6008/